In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('/content/Rice_MSC_Dataset.xlsx', sheet_name='Rice_MSC_Dataset')
df.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,ALLdaub4L,ALLdaub4a,ALLdaub4b,ALLdaub4Y,ALLdaub4Cb,ALLdaub4Cr,ALLdaub4XX,ALLdaub4YY,ALLdaub4ZZ,CLASS
0,7805,437.915,209.8215,48.0221,0.9735,99.6877,0.9775,7985,0.3547,4.3693,...,113.9924,65.0610,59.5989,104.8552,67.8779,63.0828,0.3673,0.3793,0.4733,Basmati
1,7503,340.757,138.3361,69.8417,0.8632,97.7400,0.9660,7767,0.6637,1.9807,...,105.7055,64.3685,62.2084,96.8375,65.5371,63.5832,0.3014,0.3144,0.3641,Arborio
2,5124,314.617,141.9803,46.5784,0.9447,80.7718,0.9721,5271,0.4760,3.0482,...,109.7155,62.6423,58.7439,100.2352,68.9753,59.8342,0.3233,0.3445,0.4448,Jasmine
3,7990,437.085,201.4386,51.2245,0.9671,100.8622,0.9659,8272,0.6274,3.9325,...,116.5405,64.9069,60.2562,107.2560,67.3298,63.2237,0.3880,0.4020,0.4904,Basmati
4,7433,342.893,140.3350,68.3927,0.8732,97.2830,0.9831,7561,0.6006,2.0519,...,107.7502,64.7071,61.3549,98.8704,66.2048,63.5378,0.3184,0.3303,0.3928,Arborio


In [3]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 107 columns):
 #    Column         Non-Null Count  Dtype  
---   ------         --------------  -----  
 0    AREA           75000 non-null  int64  
 1    PERIMETER      75000 non-null  float64
 2    MAJOR_AXIS     75000 non-null  float64
 3    MINOR_AXIS     75000 non-null  float64
 4    ECCENTRICITY   75000 non-null  float64
 5    EQDIASQ        75000 non-null  float64
 6    SOLIDITY       75000 non-null  float64
 7    CONVEX_AREA    75000 non-null  int64  
 8    EXTENT         75000 non-null  float64
 9    ASPECT_RATIO   75000 non-null  float64
 10   ROUNDNESS      75000 non-null  float64
 11   COMPACTNESS    75000 non-null  float64
 12   SHAPEFACTOR_1  75000 non-null  float64
 13   SHAPEFACTOR_2  75000 non-null  float64
 14   SHAPEFACTOR_3  75000 non-null  float64
 15   SHAPEFACTOR_4  75000 non-null  float64
 16   meanRR         75000 non-null  float64
 17   meanRG         75000 non-null

In [4]:
df.dropna(axis=0, inplace=True)

In [5]:
df.CLASS.value_counts()

,count
CLASS,
Basmati,15000
Arborio,15000
Karacadag,15000
Jasmine,14998
Ipsala,14994


In [6]:
scaler = StandardScaler()
x = df.drop('CLASS', axis=1)
y = df.CLASS
x = scaler.fit_transform(x)

In [7]:
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y.values.reshape(-1, 1) )

In [8]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2, random_state=2)

In [9]:
model = Sequential()

model.add(Dense(256, activation='relu', input_shape = (xtrain.shape[1],) ))
model.add(Dropout(0.3))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(5, activation='softmax'))

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
callback = EarlyStopping(monitor='val_loss',
                          patience=5,
                          min_delta=0.0001,
                          verbose=1)

In [12]:
model.fit(xtrain, ytrain, epochs=50, batch_size=32, validation_split=.2, verbose=1, callbacks=callback)

Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.9591 - loss: 0.1281 - val_accuracy: 0.9962 - val_loss: 0.0135
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9962 - loss: 0.0153 - val_accuracy: 0.9982 - val_loss: 0.0056
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9976 - loss: 0.0119 - val_accuracy: 0.9981 - val_loss: 0.0053
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9971 - loss: 0.0118 - val_accuracy: 0.9949 - val_loss: 0.0187
Epoch 5/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9975 - loss: 0.0108 - val_accuracy: 0.9980 - val_loss: 0.0092
Epoch 6/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9979 - loss: 0.0076 - val_accuracy: 0.9987 - val_loss: 0.0053
Epoch 7/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9975 - loss: 0.0100 - val_accuracy: 0.9988 - val_loss: 0.0045
Epoch 8/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9981 - loss: 0.0085 

In [13]:
from sklearn.metrics import classification_report, accuracy_score

In [14]:
ytest_pred = model.predict(xtest)

469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [15]:
accuracy_score(ytest.argmax(axis=1), ytest_pred.argmax(axis=1))

0.9991999466631108

In [16]:
print(classification_report(ytest.argmax(axis=1), ytest_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2947
           1       1.00      1.00      1.00      3005
           2       1.00      1.00      1.00      3022
           3       1.00      1.00      1.00      2982
           4       1.00      1.00      1.00      3043

    accuracy                           1.00     14999
   macro avg       1.00      1.00      1.00     14999
weighted avg       1.00      1.00      1.00     14999

